# PCA（Principal Component Analysis 主成分分析）
建立模型分析特征数据时，很可能会面临特征数据维度过大的问题。例如，根据已有的信用卡持有人信息及其违约数据来建立信用卡违约预测模型时，数据可能包含申请人的收入、年龄、性别、婚姻状况、工作单位等数百个维度的特征。如果将所有特征数据都用来拟合模型，会提高模型的复杂度，造成过拟合风险显著增大，且不同的特征数据间可能存在共线性。此时就需要对数据进行降维，以浓缩特征向量。

## 数据降维
如果特征变量的数量非常多（如成百上千个特征变量），我们往往需要进行数据降维。降维的方法主要有选择特征和抽取特征两种：选择特征是从原有的特征中挑选出最佳的特征；抽取特征则是将数据由高维向低维投影，进行坐标的线性转换。**PCA即为典型的抽取特征的方法，它不仅是对高维数据进行降维，更重要的是经过降维去除噪声，发现数据中的模式**。

### 一、 PCA的基本原理
#### 1、二维空间降维
总结来说，二维到一维的数据降维的本质就是将原始数据做零均值化处理后，寻找如下所示的合适的线性组合系数α和β，将二维数据转换为一维数据。

$X^`=αX + βY$

#### 2、n维空间降维
如果原特征变量有n个，那么就是n维空间降维。n维空间降维的思路和二维空间降维的思路是一致的，都是寻找合适的线性组合系数。例如，将n维数据（X1，X2，…，Xn）转换为一维数据，就是寻找如下所示的线性组合系数a1，a2，…，an。

$F_1 = a_1X_1 + a_2X_2 + …… + a_nX_n$

将n维数据（X1，X2，…，Xn）转换成k维数据（F1，F2，…，Fk），即将n个特征降维成k个新特征，其本质也是一样的，只是从构造1个线性组合变为构造k个线性组合，如下所示。

$F_1 = a_11X_1 + a_12X_2 + …… + a_1nX_n$

$F_1 = a_21X_1 + a_22X_2 + …… + a_2nX_n$

……

$F_1 = a_k1X_1 + a_k2X_2 + …… + a_knX_n$

简单描述一下这些系数的线性代数推导过程：将原始n维特征数据按行做零均值化处理后求得n维协方差矩阵，计算协方差矩阵的特征值及其对应的单位特征向量。将特征向量按特征值从大到小排序，使用前k行组成的矩阵乘以原始n维数据矩阵，即可得到降维后的k维数据矩阵，再通过该矩阵就能获得线性组合系数，这些线性组合系数能最大化样本方差，使新的k个特征互不相关。

实际应用中，我们只需要明白核心逻辑，不需要深究背后的数学原理，因为我们可以直接利用Python提供的相应计算库，快速计算出这些系数，无须手工计算。

### 三、PCA的代码实现

#### 1、二维空间降维的代码实现

In [1]:
# 构造例子数据
import numpy as np  
x = np.array([[1, 1], [2, 2], [3, 3]])
x

array([[1, 1],
       [2, 2],
       [3, 3]])

In [3]:
# 搭建PCA模型进行降维
from sklearn.decomposition import PCA # 引入Scikit-Learn库中的PCA模型
pca = PCA(n_components=1) # 设置PCA模型的参数n_components，即保留的成分个数，这里设置其值为1，也就是将二维数据降为一维数据
pca.fit(x) # 用fit()函数进行模型训练
x_transformed = pca.transform(x) # 用transform()函数对原始数据进行降维并将结果赋给变量X_transformed
x_transformed

array([[-1.41421356],
       [ 0.        ],
       [ 1.41421356]])

> n_components参数不仅可以设置成降维后成分的个数，还可以设置成降维后保留的信息的百分比，例如，将其设置成0.9就是在降维后保留原特征90%的信息。需要注意的是，如果将n_components参数设置成降维后成分的个数，其值不能大于min（n_samples,n_features），即样本数和特征变量数两者之中的最小数。

虽然变量x_transformed的数据格式仍然是二维数组，但其只有一列，实质上是一个一维数组，与上一小节的降维结果$\sqrt {2}$、0、$\sqrt {2}$是一致的，这样PCA降维就完成了。

In [4]:
# 数据降维其实是通过线性组合完成的，通过如下代码可以获取线性组合系数
pca.components_

array([[0.70710678, 0.70710678]])

In [5]:
# 将公式打印出来，需要注意的是，pca.components_是一个二维数组，所以要通过两层索引来获取其中的元素，且因为元素是数字，所以在进行字符串拼接时需要先用str()函数转换为字符串
a = pca.components_[0][0]
b = pca.components_[0][1]
print(str(a) + ' * X + ' +  str(b) + ' * Y')

0.7071067811865476 * X + 0.7071067811865475 * Y


#### 3、三维空间降维的代码实现
在根据已有的信用卡持有人信息及其违约数据建立信用卡违约预测模型时，数据可能包含申请人的收入、年龄、性别、婚姻状况、工作单位等数百个维度的特征。这里选择年龄（岁）、负债比率、月收入（元）3个维度的特征，使用PCA进行降维。

In [7]:
# 使用pandas库构建待降维的三维数据
import pandas as pd
x = pd.DataFrame([[45, 0.8, 9120], [40, 0.12, 2600], [38, 0.09, 3042], [30, 0.04, 3300], [39, 0.21, 3500]], columns=['年龄(岁)', '负债比率', '月收入(元)'])
x.head()

,年龄(岁),负债比率,月收入(元)
0,45,0.80,9120
1,40,0.12,2600
2,38,0.09,3042
3,30,0.04,3300
4,39,0.21,3500


可以看到，3个维度的特征数据的量级相差较大。如果数据中某一特征的数值很大，那么它在计算中所占的比重就会很大，PCA降维时会更看重这个特征，而忽略其他数值小的特征。因此，先利用数据标准化知识对3个维度的特征数据进行标准化，这也有利于PCA降维时梯度下降法的收敛，代码如下。

In [8]:
from sklearn.preprocessing import StandardScaler # 从Scikit-Learn库中引入StandardScaler()函数
x_new = StandardScaler().fit_transform(x) # 用该函数对变量X进行标准化，并将结果赋给变量x_new
x_new

array([[ 1.36321743,  1.96044639,  1.98450514],
       [ 0.33047695, -0.47222431, -0.70685302],
       [-0.08261924, -0.57954802, -0.52440206],
       [-1.73500401, -0.75842087, -0.41790353],
       [ 0.12392886, -0.15025319, -0.33534653]])

第1列对应标准化后的特征维度X，第2列对应标准化后的特征维度Y，第3列对应标准化后的特征维度Z.

In [10]:
# 搭建PCA模型进行降维
from sklearn.decomposition import PCA # 引入Scikit-Learn库中的PCA模型
pca = PCA(n_components=2) # 设置PCA模型的参数n_components为2，即将三维数据降为二维数据
pca.fit(x_new) # 用fit()函数对标准化后的数据进行模型训练
x_transformed = pca.transform(x_new) # 用transform()函数对标准化后的数据进行降维，并将结果赋给变量x_transformed
x_transformed

array([[ 3.08724247,  0.32991205],
       [-0.52888635, -0.74272137],
       [-0.70651782, -0.33057258],
       [-1.62877292,  1.05218639],
       [-0.22306538, -0.30880449]])

In [11]:
# n维数据降维也是通过线性组合完成的，通过如下代码可以获取线性组合系数
pca.components_

array([[ 0.52952108,  0.61328179,  0.58608264],
       [-0.82760701,  0.22182579,  0.51561609]])

可以看到，pca.components_是一个二维数组，第1个元素中的3个数对应的是下述公式中的系数a11、a12、a13，第2个元素中的3个数对应的是下述公式中的系数a21、a22、a23。

In [13]:
# 通过如下代码打印降维过程中原始特征的线性组合公式
dim = ['年龄(岁)', '负债比率', '月收入(元)'] # 变量dim存储特征变量名称
for i in pca.components_: # 循环遍历系数的二维数组，其中的i代表系数列表[a11，a12，a13]和[a21，a22，a23]
    formula = [] # 创建空列表formula，用于存储之后的线性组合公式
    for j in range(len(i)): # 遍历系数列表中的每一个系数，如第1个系数列表中的a11、a12、a13
        formula.append(str(i[j]) + ' * ' + dim[j]) # 通过字符串拼接的方法拼接系数、乘号和特征变量名称，并用append()函数添加到formula列表中
    print(" + ".join(formula)) # 打印线性组合公式，用join()函数将列表转换成字符串，并用“+”号连接

0.5295210839165538 * 年龄(岁) + 0.6132817922410682 * 负债比率 + 0.5860826434841948 * 月收入(元)
-0.8276070105929829 * 年龄(岁) + 0.22182579193360938 * 负债比率 + 0.5156160917294705 * 月收入(元)
